<a href="https://colab.research.google.com/github/SergheiMihailov/ml-project-cassava/blob/main/tuning_densenet_169_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
# Installs
!pip install -q -U keras-tuner

In [22]:
# Imports
import gdown
import os
import json
import csv   
import cv2
import numpy as np
import pandas as pd
from PIL import Image
import scipy.misc
import sklearn
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import tensorflow.keras.layers.experimental.preprocessing as keras_preproc
import kerastuner as kt
import keras
import io
import json
import pandas as pd
import tensorflow as tf
import sklearn
import matplotlib.pyplot as plt
import itertools
import numpy as np
import os, shutil
from datetime import datetime
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from collections import Counter
import random

In [23]:
TUNING_TRAIN_IMAGES_PATH = 'tuning_train_images/'
TUNING_TRAIN_LABELS_PATH = 'tuning_train_data.csv'
VALIDATION_IMAGES_PATH = 'val_images/'
VALIDATION_LABELS_PATH = 'val_data.csv'

LABEL_TO_DISEASE_MAP_PATH = 'label_num_to_disease_map.json'
MODEL_CP_PATH = '/content/drive/MyDrive/ml_project/models/'
HYPERPARAMETERS_PATH = '/content/drive/MyDrive/ml_project/hyperparams'

N_CLASSES = 5

# Images
IMG_SIZE = 224
SIZE = (IMG_SIZE,IMG_SIZE)
INPUT_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

# Training
EPOCHS_TO_TRAIN = 30
BATCH_SIZE = 32

In [24]:
os.chdir('/content/')
# Downloading data
if not os.path.exists(LABEL_TO_DISEASE_MAP_PATH):
  !wget files.brainfriz.com/ml_project/label_num_to_disease_map.json

if not os.path.exists(TUNING_TRAIN_IMAGES_PATH): 
  !wget files.brainfriz.com/ml_project/tuning_data.rar
  
if not os.path.exists('/content/drive/MyDrive/ml_project/'):
  !mkdir /content/drive/MyDrive/ml_project

if not os.path.exists(HYPERPARAMETERS_PATH):
  !mkdir /content/drive/MyDrive/ml_project/hyperparams

In [25]:
!unrar x tuning_data.rar > /dev/null




Would you like to replace the existing file val_images/1010648150.jpg
137071 bytes, modified on 2021-03-24 06:54
with a new one
137071 bytes, modified on 2021-03-24 06:54

[Y]es, [N]o, [A]ll, n[E]ver, [R]ename, [Q]uit ^C


In [26]:
!rm -f tuning_train.rar

In [27]:
# Dataset
train_data = pd.read_csv(TUNING_TRAIN_LABELS_PATH)
validation_data = pd.read_csv(VALIDATION_LABELS_PATH)

f = open(LABEL_TO_DISEASE_MAP_PATH)
label_class_name = json.load(f)
label_class_name = {int(k):v for k,v in label_class_name.items()}

train_data['class_name'] = train_data.label.map(label_class_name)
validation_data['class_name'] = validation_data.label.map(label_class_name)

train_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

proportion_data_used_for_tuning = 0.2

if proportion_data_used_for_tuning:
  train_indices = list(range(len(train_data)))
  random.shuffle(train_indices)
  train_indices = train_indices[:int(proportion_data_used_for_tuning*len(train_indices))]
  tuning_data = train_data.iloc[train_indices]

  print(Counter(tuning_data['class_name']))

train_gen = train_datagen.flow_from_dataframe(tuning_data,
                                directory = TUNING_TRAIN_IMAGES_PATH,
                                x_col = 'image_id',
                                y_col = 'class_name',
                                color_mode='rgb',
                                class_mode='categorical',
                                target_size = SIZE,
                                shuffle = True,
                                batch_size = BATCH_SIZE)

val_gen = test_datagen.flow_from_dataframe(validation_data,
                                directory = VALIDATION_IMAGES_PATH,
                                x_col = 'image_id',
                                y_col = 'class_name',
                                color_mode='rgb',
                                class_mode='categorical',
                                target_size = SIZE,
                                shuffle = False,
                                batch_size = BATCH_SIZE)

Counter({'Cassava Mosaic Disease (CMD)': 2635, 'Cassava Bacterial Blight (CBB)': 2546, 'Cassava Green Mottle (CGM)': 2537, 'Cassava Brown Streak Disease (CBSD)': 2530, 'Healthy': 2510})
Found 12758 validated image filenames belonging to 5 classes.
Found 1000 validated image filenames belonging to 5 classes.


In [29]:
# Models definition

def getEfficientNetB0():
  return [
      # architecture
      tf.keras.applications.EfficientNetB0(
        include_top=True, weights=None, input_tensor=None,
        input_shape=INPUT_SHAPE, pooling=None, classes=N_CLASSES,
        classifier_activation='softmax', drop_connect_rate=0.4
      ),
      # preprocess_input
      tf.keras.applications.efficientnet.preprocess_input
  ]

def getResNet50(): 
  return [
      # architecture 
      tf.keras.applications.ResNet50(
        include_top=True, weights=None, input_tensor=None,
        input_shape=INPUT_SHAPE, pooling='avg', classes=N_CLASSES,
        classifier_activation='softmax'
      ),
      # preprocess_input
      tf.keras.applications.resnet_v2.preprocess_input
  ]

def getMobileNetV3Small(): 
  return [
      # architecture
      tf.keras.applications.MobileNetV3Small(
        input_shape=INPUT_SHAPE, alpha=1, minimalistic=True, include_top=True,
        weights=None, input_tensor=None, classes=5, pooling=None,
        dropout_rate=0, classifier_activation='softmax'
      ),
      # preprocess_input:
      tf.keras.applications.mobilenet_v3.preprocess_input
  ]


def getDenseNet169(): 
  return [
      # architecture
      tf.keras.applications.DenseNet169(
          include_top=True, weights=None, input_tensor=None,
          input_shape=None, pooling=None, classes=5
      ),
      # preprocess_input:
      tf.keras.applications.densenet.preprocess_input
  ]

def model_builder(hp):
  inputs = tf.keras.layers.Input(shape=INPUT_SHAPE)
  architecture, preprocess_input = getResNet50() # change architecture here
  #input_layer = preprocess_input(tf.keras.layers.Input(shape=INPUT_SHAPE)) #only for mobilenet change inputs to input_layer
  model = tf.keras.Model(inputs, architecture(inputs))
  
  hp_learning_rate = hp.Float('learning_rate', 1e-05, 1, sampling='log')
  hp_label_smoothing = hp.Float('label_smoothing', 1e-09, 1e-02, sampling='log')

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate, epsilon=0.1),
                loss=keras.losses.CategoricalCrossentropy(label_smoothing=hp_label_smoothing),
                metrics=['accuracy'])

  return model



In [31]:
tuner =None
best_hps=None

In [ ]:
# Hyperparameter tuning
MAX_EPOCHS=15

tuning_callbacks = [
  tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3),
  tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,patience=2, min_lr=0.00001),
]

tuner = kt.Hyperband(model_builder, # make sure to change the model in the model_builder function.
                     objective='val_accuracy',
                     max_epochs=MAX_EPOCHS,
                     factor=3,
                     directory=HYPERPARAMETERS_PATH+'max_epochs_'+str(MAX_EPOCHS),
                     project_name='cassava')

tuner.search(train_gen, epochs=EPOCHS_TO_TRAIN, validation_data=val_gen, callbacks=tuning_callbacks)

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters()[0]
print(f"""
The hyperparameter search is complete. Optimal values: \n
learning_rate: {best_hps.get('learning_rate')}; \n
label_smoothing: {best_hps.get('label_smoothing')}; \n
""")

model = tuner.get_best_models(num_models=1)[0]
model_json = model.to_json()
with open(MODEL_CP_PATH+"tuning_model_max_epochs+"+str(MAX_EPOCHS)+".json", "w+") as json_file:
  json_file.write(model_json)

model.save_weights(MODEL_CP_PATH+"tuning_model_max_epochs+"+str(MAX_EPOCHS)+".h5")

Trial 18 Complete [00h 25m 16s]
val_accuracy: 0.5070000290870667

Best val_accuracy So Far: 0.5070000290870667
Total elapsed time: 02h 23m 01s

Search: Running Trial #19

Hyperparameter    |Value             |Best Value So Far 
learning_rate     |0.00078431        |0.02292           
label_smoothing   |0.00013733        |1.019e-07         
tuner/epochs      |5                 |15                
tuner/initial_e...|0                 |5                 
tuner/bracket     |1                 |2                 
tuner/round       |0                 |2                 

Epoch 1/5
212/399 [==============>...............] - ETA: 1:06 - loss: 1.6276 - accuracy: 0.2689

In [35]:
model = tuner.get_best_models(num_models=1)
model_json = model.to_json()
with open(MODEL_CP_PATH+"tuning_model.json", "w+") as json_file:
  json_file.write(model_json)

model.save_weights(MODEL_CP_PATH+"tuning_model.h5")

NameError: ignored